# Hybrid Search: dense and sparse vectors

LlamaIndex integration with Qdrant supports sparse embeddings as well. From the user perspective, it doesn't change much, as they interact through the same interface. Since sparse and dense vectors work best in different setups, it makes sense to combine them if we want to have the best of both worlds. There are, however, some parameters we can control.

Let's again start with recreating our pipeline, but this time we will use the other collection that has sparse vectors as well.


In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
    embed_model="local:BAAI/bge-large-en"
)

In [ ]:
from qdrant_client import QdrantClient
from llama_index.vector_stores.qdrant import QdrantVectorStore

import os

client = QdrantClient(
    os.environ.get("QDRANT_URL"), 
    api_key=os.environ.get("QDRANT_API_KEY"),
)
vector_store_hybrid = QdrantVectorStore(
    client=client,
    collection_name="hacker-news-hybrid",
    enable_hybrid=True,
    batch_size=20,  # this is important for the ingestion
)

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store_hybrid,
    service_context=service_context,
)

## Differences between sparse and dense vectors

Sparse vectors are usually used in high-dimensional spaces, where the majority of the elements are zero. A single dimension represents a single word, so the dimensionality of the space is equal to the size of the vocabulary, with just a few non-zero values. 

There are various ways to create sparse vectors, but the most common one is to use the TF-IDF or BM25 representation. It's a simple and effective way to represent the importance of words in a document and in many cases create a solid baseline for the search.

LlamaIndex uses SPLADE by default, which is based on transformers, similar to dense embedding models. **The main advantage of using sparse vectors is that they overcome the problem of vocabulary mismatch**. If a word is not present in the vocabulary of the dense embedding model, we can still represent it using the sparse vectors.

## Using sparse vectors only

Before we dive into the hybrid search, let's see what might be achieved by using sparse vectors alone. We already know the nodes retrieved by dense vectors so it makes sense to compare the results returned by both methods.

In [ ]:
from llama_index.vector_stores.types import VectorStoreQueryMode
from llama_index.indices.vector_store import VectorIndexRetriever

sparse_retriever = VectorIndexRetriever(
    index=index,
    vector_store_query_mode=VectorStoreQueryMode.SPARSE,
    sparse_top_k=5,
)

In [ ]:
nodes = sparse_retriever.retrieve("What is the best way to learn programming?")
for i, node in enumerate(nodes):
    print(i + 1, node.text, end="\n\n")

## Hybrid search

There are some specific use cases in which we may prefer to use just the sparse vectors. But both methods may complement each other and we usually need to find the sweet spot. The `VectorIndexRetriever` class allows us to control the parameters of the search. We can set the `sparse_top_k` and `similarity_top_k` parameters to control the number of results returned by each method. We can also set the `alpha` parameters to control the importance of each method (`0.0` = sparse, `1.0` = dense vectors only).

In [ ]:
hybrid_retriever = VectorIndexRetriever(
    index=index,
    vector_store_query_mode=VectorStoreQueryMode.HYBRID,
    sparse_top_k=5,
    similarity_top_k=5,
    alpha=0.1,
)

In [ ]:
nodes = hybrid_retriever.retrieve("What is the best way to learn programming?")
for i, node in enumerate(nodes):
    print(i + 1, node.text, end="\n\n")

In [ ]:
# We shouldn't be modifying the alpha parameter after the retriever has been created
# but that's the easiest way to show the effect of the parameter
hybrid_retriever._alpha = 0.9

nodes = hybrid_retriever.retrieve("What is the best way to learn programming?")
for i, node in enumerate(nodes):
    print(i + 1, node.text, end="\n\n")